In [ ]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313501 sha256=7ac78db8f1d21ca88e03d603d042af0934cd0f893807a1eebf7d4598fbd601b1
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!wget -O en-hi.txt https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.txt

--2025-04-01 17:59:26--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.169.121.110, 3.169.121.57, 3.169.121.81, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.169.121.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930856 (909K) [text/x-c++]
Saving to: ‘en-hi.txt’

en-hi.txt           100%[===================>] 909.04K  1.85MB/s    in 0.5s    

2025-04-01 17:59:27 (1.85 MB/s) - ‘en-hi.txt’ saved [930856/930856]



In [ ]:
!ls -lh en-hi.txt


-rw-r--r-- 1 root root 910K Jan 31  2019 en-hi.txt


In [ ]:
!head -3 "/content/drive/MyDrive/MUSE/wiki.hi.vec"


158016 300
</s> 0.11805 0.020802 -0.1089 -0.15894 -0.099375 0.18186 0.10915 -0.13444 0.088495 0.13883 0.0087136 0.000868 -0.15426 -0.11779 -0.19404 -0.09652 -0.0016641 0.018956 -0.061415 -0.067655 0.096204 0.11215 -0.032286 0.47692 0.30003 -0.045058 0.071621 -0.03294 0.15929 -0.20687 -0.116 0.11844 -0.04063 -0.11685 0.05468 0.011329 0.10962 -0.12171 -0.030407 -0.048362 0.12066 -0.089797 0.037743 -0.17441 0.16022 0.0015027 0.082437 -0.11796 0.14972 0.043381 0.010314 -0.039055 -0.16809 -0.075393 0.064941 0.016056 -0.022792 -0.0080583 0.13621 -0.070592 0.18616 0.24617 0.16976 0.14741 0.1235 -0.12475 -0.11184 0.14197 -0.24451 0.011281 0.23877 -0.14358 0.081527 0.061886 0.043097 0.15992 0.023049 -0.085767 -0.025619 0.15381 -0.28751 0.040967 -0.18054 0.10728 0.11467 -0.0030426 0.037164 -0.053903 0.14863 -0.076724 -0.013779 -0.0013568 0.18816 0.26511 -0.10453 -0.011901 0.13636 0.0047142 0.19787 0.030574 -0.059869 0.073519 0.12941 0.0089557 -0.057192 0.22919 -0.08126 -0.049841 -0.0011047 -0.01

In [ ]:
# Install necessary libraries
!pip install numpy scipy

# Import Libraries
import numpy as np
import os
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import cdist
from google.colab import drive


# Step 2: Define File Paths (Modify these paths if needed)
EN_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.en.vec"  # English embeddings
HI_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.hi.vec"  # Hindi embeddings
LEXICON_PATH = "en-hi.txt"  # We already downloaded this in Colab

# Step 3: Load Pre-trained Word Embeddings
def load_muse_embeddings(file_path, max_vocab=50000):
    word2vec = {}
    with open(file_path, "r", encoding="utf-8") as f:
        first_line = f.readline()  # Skip header line
        for i, line in enumerate(f):
            if i >= max_vocab:
                break
            parts = line.rstrip().split(" ")
            word = parts[0]
            try:
                vec = np.array(parts[1:], dtype=np.float32)  # Convert only the vector part
                word2vec[word] = vec
            except ValueError:
                print(f"Skipping malformed line {i+2}: {line}")
    return word2vec

# Step 4: Load Bilingual Lexicon
def load_bilingual_lexicon(file_path, max_pairs=10000):
    lexicon = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_pairs:
                break
            en_word, hi_word = line.strip().split()
            lexicon.append((en_word, hi_word))
    return lexicon

#  Step 5: Extract Embeddings for Words in the Lexicon
def extract_embedding_matrix(lexicon, src_embeddings, tgt_embeddings):
    src_vectors, tgt_vectors = [], []
    valid_pairs = []
    for en_word, hi_word in lexicon:
        if en_word in src_embeddings and hi_word in tgt_embeddings:
            src_vectors.append(src_embeddings[en_word])
            tgt_vectors.append(tgt_embeddings[hi_word])
            valid_pairs.append((en_word, hi_word))  # Keep only valid pairs
    return np.array(src_vectors), np.array(tgt_vectors), valid_pairs

# Step 6: Perform Procrustes Alignment
def procrustes_alignment(X, Y):
    W, _ = orthogonal_procrustes(X, Y)  # Find optimal rotation matrix
    return W

# Step 7: Find Nearest Neighbors
def nearest_neighbors(src_emb, tgt_emb, k=5):
    distances = cdist(src_emb, tgt_emb, metric='cosine')  # Compute cosine distances
    return np.argsort(distances, axis=1)[:, :k]  # Get k nearest indices

# Step 8: Evaluate Translation Accuracy
def evaluate_translation(valid_pairs, src_embeddings, tgt_embeddings, W, top_k=5):
    src_words, tgt_words = zip(*valid_pairs)

    # Get embeddings for the valid word pairs
    src_emb = np.array([src_embeddings[w] for w in src_words])
    tgt_emb = np.array([tgt_embeddings[w] for w in tgt_words])

    # Apply alignment transformation
    aligned_src_emb = src_emb @ W

    # Get nearest neighbors
    nn_indices = nearest_neighbors(aligned_src_emb, tgt_emb, k=top_k)

    # Compute Precision@1 and Precision@5
    precision_at_1 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:1]] for i in range(len(tgt_words))])
    precision_at_5 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:5]] for i in range(len(tgt_words))])


    return precision_at_1, precision_at_5

# Step 9: Run the Entire Pipeline
if __name__ == "__main__":
    # Load Pre-trained MUSE Embeddings
    print("Loading embeddings...")
    en_embeddings = load_muse_embeddings(EN_EMBEDDINGS_PATH)
    hi_embeddings = load_muse_embeddings(HI_EMBEDDINGS_PATH)

    # Load Bilingual Lexicon
    print("Loading bilingual lexicon...")
    lexicon = load_bilingual_lexicon(LEXICON_PATH, max_pairs=10000)

    # Extract embeddings for words in the lexicon
    print("Extracting embeddings for alignment...")
    X, Y, valid_pairs = extract_embedding_matrix(lexicon, en_embeddings, hi_embeddings)

    # Perform Procrustes Alignment
    print("Performing Procrustes alignment...")
    W = procrustes_alignment(X, Y)

    # Evaluate translation accuracy
    print("Evaluating translation accuracy...")
    p_at_1, p_at_5 = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W)

    # Print Results
    print(f"Precision@1: {p_at_1 * 100:.2f}%")
    print(f"Precision@5: {p_at_5 * 100:.2f}%")


Loading embeddings...
Loading bilingual lexicon...
Extracting embeddings for alignment...
Performing Procrustes alignment...
Evaluating translation accuracy...
Precision@1: 42.53%
Precision@5: 69.71%


In [ ]:
from scipy.spatial.distance import cosine

def compute_cosine_similarity(src_emb, tgt_emb, W):
    # Apply alignment transformation to the source embeddings
    aligned_src_emb = src_emb @ W

    # Compute cosine similarities between aligned source embeddings and target embeddings
    similarities = np.array([[1 - cosine(src, tgt) for tgt in tgt_emb] for src in aligned_src_emb])
    return similarities

# Adding this to the pipeline after evaluating translation accuracy
if __name__ == "__main__":
    # Existing code...
    # Evaluate translation accuracy
    print("Evaluating translation accuracy...")
    p_at_1, p_at_5 = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W)

    # Compute and display cosine similarities
    print("Computing cosine similarities...")
    src_words, tgt_words = zip(*valid_pairs)
    src_emb = np.array([en_embeddings[w] for w in src_words])
    tgt_emb = np.array([hi_embeddings[w] for w in tgt_words])
    similarities = compute_cosine_similarity(src_emb, tgt_emb, W)

    # Optionally print a few similarity results
    print(f"Sample cosine similarities: {similarities[:5]}")


Evaluating translation accuracy...
Computing cosine similarities...
Sample cosine similarities: [[0.7445441  0.39032638 0.41594315 ... 0.2965145  0.28866065 0.23254883]
 [0.36138368 0.6293223  0.54573894 ... 0.26155645 0.23415339 0.22959673]
 [0.36138368 0.6293223  0.54573894 ... 0.26155645 0.23415339 0.22959673]
 [0.4783002  0.33716756 0.29790092 ... 0.18006063 0.28276563 0.2231099 ]
 [0.5009129  0.4919359  0.4435017  ... 0.15416837 0.262429   0.30277038]]


In [ ]:
# Install necessary libraries
!pip install numpy scipy

# Import Libraries
import numpy as np
import os
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import cdist
from google.colab import drive


#  Step 2: Define File Paths (Modify these paths if needed)
EN_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.en.vec"  # English embeddings
HI_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.hi.vec"  # Hindi embeddings
LEXICON_PATH = "en-hi.txt"  # We already downloaded this in Colab

#  Step 3: Load Pre-trained Word Embeddings
def load_muse_embeddings(file_path, max_vocab=50000):
    word2vec = {}
    with open(file_path, "r", encoding="utf-8") as f:
        first_line = f.readline()  # Skip header line
        for i, line in enumerate(f):
            if i >= max_vocab:
                break
            parts = line.rstrip().split(" ")
            word = parts[0]
            try:
                vec = np.array(parts[1:], dtype=np.float32)  # Convert only the vector part
                word2vec[word] = vec
            except ValueError:
                print(f"Skipping malformed line {i+2}: {line}")
    return word2vec

#  Step 4: Load Bilingual Lexicon
def load_bilingual_lexicon(file_path, max_pairs=10000):
    lexicon = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_pairs:
                break
            en_word, hi_word = line.strip().split()
            lexicon.append((en_word, hi_word))
    return lexicon

#  Step 5: Extract Embeddings for Words in the Lexicon
def extract_embedding_matrix(lexicon, src_embeddings, tgt_embeddings):
    src_vectors, tgt_vectors = [], []
    valid_pairs = []
    for en_word, hi_word in lexicon:
        if en_word in src_embeddings and hi_word in tgt_embeddings:
            src_vectors.append(src_embeddings[en_word])
            tgt_vectors.append(tgt_embeddings[hi_word])
            valid_pairs.append((en_word, hi_word))  # Keep only valid pairs
    return np.array(src_vectors), np.array(tgt_vectors), valid_pairs

#  Step 6: Perform Procrustes Alignment
def procrustes_alignment(X, Y):
    W, _ = orthogonal_procrustes(X, Y)  # Find optimal rotation matrix
    return W

#  Step 7: Find Nearest Neighbors
def nearest_neighbors(src_emb, tgt_emb, k=5):
    distances = cdist(src_emb, tgt_emb, metric='cosine')  # Compute cosine distances
    return np.argsort(distances, axis=1)[:, :k]  # Get k nearest indices

#  Step 8: Evaluate Translation Accuracy
def evaluate_translation(valid_pairs, src_embeddings, tgt_embeddings, W, top_k=5):
    src_words, tgt_words = zip(*valid_pairs)

    # Get embeddings for the valid word pairs
    src_emb = np.array([src_embeddings[w] for w in src_words])
    tgt_emb = np.array([tgt_embeddings[w] for w in tgt_words])

    # Apply alignment transformation
    aligned_src_emb = src_emb @ W

    # Get nearest neighbors
    nn_indices = nearest_neighbors(aligned_src_emb, tgt_emb, k=top_k)

    # Compute Precision@1 and Precision@5
    precision_at_1 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:1]] for i in range(len(tgt_words))])
    precision_at_5 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:5]] for i in range(len(tgt_words))])


    return precision_at_1, precision_at_5

#  Step 9: Run the Entire Pipeline
if __name__ == "__main__":
    # Load Pre-trained MUSE Embeddings
    print("Loading embeddings...")
    en_embeddings = load_muse_embeddings(EN_EMBEDDINGS_PATH)
    hi_embeddings = load_muse_embeddings(HI_EMBEDDINGS_PATH)

    # Load Bilingual Lexicon
    print("Loading bilingual lexicon...")
    lexicon = load_bilingual_lexicon(LEXICON_PATH, max_pairs=5000)

    # Extract embeddings for words in the lexicon
    print("Extracting embeddings for alignment...")
    X, Y, valid_pairs = extract_embedding_matrix(lexicon, en_embeddings, hi_embeddings)

    # Perform Procrustes Alignment
    print("Performing Procrustes alignment...")
    W = procrustes_alignment(X, Y)

    # Evaluate translation accuracy
    print("Evaluating translation accuracy...")
    p_at_1, p_at_5 = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W)

    # Print Results
    print(f"Precision@1: {p_at_1 * 100:.2f}%")
    print(f"Precision@5: {p_at_5 * 100:.2f}%")


Loading embeddings...
Loading bilingual lexicon...
Extracting embeddings for alignment...
Performing Procrustes alignment...
Evaluating translation accuracy...
Precision@1: 43.76%
Precision@5: 77.73%


In [ ]:
# Install necessary libraries
!pip install numpy scipy

# Import Libraries
import numpy as np
import os
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import cdist
from google.colab import drive


# Step 2: Define File Paths (Modify these paths if needed)
EN_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.en.vec"  # English embeddings
HI_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.hi.vec"  # Hindi embeddings
LEXICON_PATH = "en-hi.txt"  # We already downloaded this in Colab

#  Step 3: Load Pre-trained Word Embeddings
def load_muse_embeddings(file_path, max_vocab=50000):
    word2vec = {}
    with open(file_path, "r", encoding="utf-8") as f:
        first_line = f.readline()  # Skip header line
        for i, line in enumerate(f):
            if i >= max_vocab:
                break
            parts = line.rstrip().split(" ")
            word = parts[0]
            try:
                vec = np.array(parts[1:], dtype=np.float32)  # Convert only the vector part
                word2vec[word] = vec
            except ValueError:
                print(f"Skipping malformed line {i+2}: {line}")
    return word2vec

# Step 4: Load Bilingual Lexicon
def load_bilingual_lexicon(file_path, max_pairs=20000):
    lexicon = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_pairs:
                break
            en_word, hi_word = line.strip().split()
            lexicon.append((en_word, hi_word))
    return lexicon

#  Step 5: Extract Embeddings for Words in the Lexicon
def extract_embedding_matrix(lexicon, src_embeddings, tgt_embeddings):
    src_vectors, tgt_vectors = [], []
    valid_pairs = []
    for en_word, hi_word in lexicon:
        if en_word in src_embeddings and hi_word in tgt_embeddings:
            src_vectors.append(src_embeddings[en_word])
            tgt_vectors.append(tgt_embeddings[hi_word])
            valid_pairs.append((en_word, hi_word))  # Keep only valid pairs
    return np.array(src_vectors), np.array(tgt_vectors), valid_pairs

# Step 6: Perform Procrustes Alignment
def procrustes_alignment(X, Y):
    W, _ = orthogonal_procrustes(X, Y)  # Find optimal rotation matrix
    return W

# Step 7: Find Nearest Neighbors
def nearest_neighbors(src_emb, tgt_emb, k=5):
    distances = cdist(src_emb, tgt_emb, metric='cosine')  # Compute cosine distances
    return np.argsort(distances, axis=1)[:, :k]  # Get k nearest indices

#  Step 8: Evaluate Translation Accuracy
def evaluate_translation(valid_pairs, src_embeddings, tgt_embeddings, W, top_k=5):
    src_words, tgt_words = zip(*valid_pairs)

    # Get embeddings for the valid word pairs
    src_emb = np.array([src_embeddings[w] for w in src_words])
    tgt_emb = np.array([tgt_embeddings[w] for w in tgt_words])

    # Apply alignment transformation
    aligned_src_emb = src_emb @ W

    # Get nearest neighbors
    nn_indices = nearest_neighbors(aligned_src_emb, tgt_emb, k=top_k)

    # Compute Precision@1 and Precision@5
    precision_at_1 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:1]] for i in range(len(tgt_words))])
    precision_at_5 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:5]] for i in range(len(tgt_words))])


    return precision_at_1, precision_at_5

# Step 9: Run the Entire Pipeline
if __name__ == "__main__":
    # Load Pre-trained MUSE Embeddings
    print("Loading embeddings...")
    en_embeddings = load_muse_embeddings(EN_EMBEDDINGS_PATH)
    hi_embeddings = load_muse_embeddings(HI_EMBEDDINGS_PATH)

    # Load Bilingual Lexicon
    print("Loading bilingual lexicon...")
    lexicon = load_bilingual_lexicon(LEXICON_PATH, max_pairs=10000)

    # Extract embeddings for words in the lexicon
    print("Extracting embeddings for alignment...")
    X, Y, valid_pairs = extract_embedding_matrix(lexicon, en_embeddings, hi_embeddings)

    # Perform Procrustes Alignment
    print("Performing Procrustes alignment...")
    W = procrustes_alignment(X, Y)

    # Evaluate translation accuracy
    print("Evaluating translation accuracy...")
    p_at_1, p_at_5 = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W)

    # Print Results
    print(f"Precision@1: {p_at_1 * 100:.2f}%")
    print(f"Precision@5: {p_at_5 * 100:.2f}%")


Loading embeddings...
Loading bilingual lexicon...
Extracting embeddings for alignment...
Performing Procrustes alignment...
Evaluating translation accuracy...
Precision@1: 42.53%
Precision@5: 69.71%


In [ ]:
# Install necessary libraries
!pip install numpy scipy

# Import Libraries
import numpy as np
import os
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import cdist
from google.colab import drive


#  Step 2: Define File Paths (Modify these paths if needed)
EN_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.en.vec"  # English embeddings
HI_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.hi.vec"  # Hindi embeddings
LEXICON_PATH = "en-hi.txt"  # We already downloaded this in Colab

#  Step 3: Load Pre-trained Word Embeddings
def load_muse_embeddings(file_path, max_vocab=50000):
    word2vec = {}
    with open(file_path, "r", encoding="utf-8") as f:
        first_line = f.readline()  # Skip header line
        for i, line in enumerate(f):
            if i >= max_vocab:
                break
            parts = line.rstrip().split(" ")
            word = parts[0]
            try:
                vec = np.array(parts[1:], dtype=np.float32)  # Convert only the vector part
                word2vec[word] = vec
            except ValueError:
                print(f"Skipping malformed line {i+2}: {line}")
    return word2vec

# Step 4: Load Bilingual Lexicon
def load_bilingual_lexicon(file_path, max_pairs=10000):
    lexicon = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_pairs:
                break
            en_word, hi_word = line.strip().split()
            lexicon.append((en_word, hi_word))
    return lexicon

#  Step 5: Extract Embeddings for Words in the Lexicon
def extract_embedding_matrix(lexicon, src_embeddings, tgt_embeddings):
    src_vectors, tgt_vectors = [], []
    valid_pairs = []
    for en_word, hi_word in lexicon:
        if en_word in src_embeddings and hi_word in tgt_embeddings:
            src_vectors.append(src_embeddings[en_word])
            tgt_vectors.append(tgt_embeddings[hi_word])
            valid_pairs.append((en_word, hi_word))  # Keep only valid pairs
    return np.array(src_vectors), np.array(tgt_vectors), valid_pairs

#  Step 6: Perform Procrustes Alignment
def procrustes_alignment(X, Y):
    W, _ = orthogonal_procrustes(X, Y)  # Find optimal rotation matrix
    return W

#  Step 7: Find Nearest Neighbors
def nearest_neighbors(src_emb, tgt_emb, k=5):
    distances = cdist(src_emb, tgt_emb, metric='cosine')  # Compute cosine distances
    return np.argsort(distances, axis=1)[:, :k]  # Get k nearest indices

#  Step 8: Evaluate Translation Accuracy
def evaluate_translation(valid_pairs, src_embeddings, tgt_embeddings, W, top_k=5):
    src_words, tgt_words = zip(*valid_pairs)

    # Get embeddings for the valid word pairs
    src_emb = np.array([src_embeddings[w] for w in src_words])
    tgt_emb = np.array([tgt_embeddings[w] for w in tgt_words])

    # Apply alignment transformation
    aligned_src_emb = src_emb @ W

    # Get nearest neighbors
    nn_indices = nearest_neighbors(aligned_src_emb, tgt_emb, k=top_k)

    # Compute Precision@1 and Precision@5
    precision_at_1 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:1]] for i in range(len(tgt_words))])
    precision_at_5 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:5]] for i in range(len(tgt_words))])


    return precision_at_1, precision_at_5

# Step 9: Run the Entire Pipeline
if __name__ == "__main__":
    # Load Pre-trained MUSE Embeddings
    print("Loading embeddings...")
    en_embeddings = load_muse_embeddings(EN_EMBEDDINGS_PATH)
    hi_embeddings = load_muse_embeddings(HI_EMBEDDINGS_PATH)

    # Load Bilingual Lexicon
    print("Loading bilingual lexicon...")
    lexicon = load_bilingual_lexicon(LEXICON_PATH, max_pairs=1000)

    # Extract embeddings for words in the lexicon
    print("Extracting embeddings for alignment...")
    X, Y, valid_pairs = extract_embedding_matrix(lexicon, en_embeddings, hi_embeddings)

    # Perform Procrustes Alignment
    print("Performing Procrustes alignment...")
    W = procrustes_alignment(X, Y)

    # Evaluate translation accuracy
    print("Evaluating translation accuracy...")
    p_at_1, p_at_5 = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W)

    # Print Results
    print(f"Precision@1: {p_at_1 * 100:.2f}%")
    print(f"Precision@5: {p_at_5 * 100:.2f}%")


Loading embeddings...
Loading bilingual lexicon...
Extracting embeddings for alignment...
Performing Procrustes alignment...
Evaluating translation accuracy...
Precision@1: 41.68%
Precision@5: 92.81%


CLS Implementation


In [ ]:
# Install necessary libraries
!pip install numpy scipy torch

# Import Libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import cdist
from scipy.linalg import orthogonal_procrustes
from google.colab import drive

# Step 1: Load Pre-trained Word Embeddings
def load_muse_embeddings(file_path, max_vocab=50000):
    word2vec = {}
    with open(file_path, "r", encoding="utf-8") as f:
        first_line = f.readline()  # Skip header line
        for i, line in enumerate(f):
            if i >= max_vocab:
                break
            parts = line.rstrip().split(" ")
            word = parts[0]
            try:
                vec = np.array(parts[1:], dtype=np.float32)  # Convert only the vector part
                word2vec[word] = vec
            except ValueError:
                print(f"Skipping malformed line {i+2}: {line}")
    return word2vec

# Step 2: Load Bilingual Lexicon
def load_bilingual_lexicon(file_path, max_pairs=10000):
    lexicon = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_pairs:
                break
            en_word, hi_word = line.strip().split()
            lexicon.append((en_word, hi_word))
    return lexicon

# Step 3: Extract Embeddings for Words in the Lexicon
def extract_embedding_matrix(lexicon, src_embeddings, tgt_embeddings):
    src_vectors, tgt_vectors = [], []
    valid_pairs = []
    for en_word, hi_word in lexicon:
        if en_word in src_embeddings and hi_word in tgt_embeddings:
            src_vectors.append(src_embeddings[en_word])
            tgt_vectors.append(tgt_embeddings[hi_word])
            valid_pairs.append((en_word, hi_word))  # Keep only valid pairs
    return np.array(src_vectors), np.array(tgt_vectors), valid_pairs

# Step 4: Perform Procrustes Alignment
def procrustes_alignment(X, Y):
    W, _ = orthogonal_procrustes(X, Y)  # Find optimal rotation matrix
    return W

# Step 5: Compute Cosine Similarity
def csls(src_emb, tgt_emb, k=5):
    """
    Compute the CSLS (Cross-Domain Similarity Local Scaling) score between source and target embeddings.
    """
    # Calculate cosine similarity between all pairs of source and target embeddings
    cosine_sim = 1 - cdist(src_emb, tgt_emb, metric='cosine')  # Cosine distance to cosine similarity

    # Compute the k nearest neighbors for each source word in target space
    src_nn_sim = np.sort(cosine_sim, axis=1)[:, -k:]  # k largest similarities for each source word
    tgt_nn_sim = np.sort(cosine_sim, axis=0)[-k:, :]  # k largest similarities for each target word

    # Compute CSLS
    csls_scores = cosine_sim - (np.mean(src_nn_sim, axis=1)[:, None] + np.mean(tgt_nn_sim, axis=0))
    return csls_scores

# Nearest neighbors using CSLS
def nearest_neighbors_csls(src_emb, tgt_emb, k=5):
    csls_scores = csls(src_emb, tgt_emb, k=k)
    return np.argsort(csls_scores, axis=1)[:, :k]  # Get k nearest indices

# Step 6: Adversarial Training (Discriminator)
class Discriminator(nn.Module):
    def __init__(self, emb_size):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(emb_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.fc(x)

def adversarial_training(src_emb, tgt_emb, epochs=100, batch_size=32, lr=0.001):
    # Convert numpy arrays to torch tensors
    src_emb = torch.tensor(src_emb, dtype=torch.float32)
    tgt_emb = torch.tensor(tgt_emb, dtype=torch.float32)

    # Initialize the discriminator
    discriminator = Discriminator(src_emb.shape[1])  # Assume source and target embeddings have the same size
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(discriminator.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        # Shuffle source and target embeddings
        perm_src = torch.randperm(src_emb.size(0))
        perm_tgt = torch.randperm(tgt_emb.size(0))

        # Create batches
        for i in range(0, len(src_emb), batch_size):
            # Create a batch for source and target embeddings
            src_batch = src_emb[perm_src[i:i + batch_size]]
            tgt_batch = tgt_emb[perm_tgt[i:i + batch_size]]

            # Concatenate the source and target batches
            inputs = torch.cat([src_batch, tgt_batch], dim=0)
            labels = torch.cat([torch.zeros(src_batch.size(0)), torch.ones(tgt_batch.size(0))], dim=0)

            # Forward pass
            outputs = discriminator(inputs)

            # Compute the loss
            loss = criterion(outputs.view(-1), labels)

            # Backward pass and update the weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    return discriminator

# Step 7: Evaluate Translation Accuracy
def evaluate_translation(valid_pairs, src_embeddings, tgt_embeddings, W=None, discriminator=None, top_k=5, method='procrustes'):
    src_words, tgt_words = zip(*valid_pairs)

    # Get embeddings for the valid word pairs
    src_emb = np.array([src_embeddings[w] for w in src_words])
    tgt_emb = np.array([tgt_embeddings[w] for w in tgt_words])

    # Apply alignment transformation
    if method == 'procrustes':
        aligned_src_emb = src_emb @ W  # Apply Procrustes alignment
    elif method == 'csls':
        aligned_src_emb = src_emb  # For CSLS, no transformation is applied
    else:
        raise ValueError("Unknown method: choose either 'procrustes' or 'csls'")

    # Get nearest neighbors
    nn_indices = nearest_neighbors_csls(aligned_src_emb, tgt_emb, k=top_k)

    # Compute Precision@1 and Precision@5
    precision_at_1 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:1]] for i in range(len(tgt_words))])
    precision_at_5 = np.mean([tgt_words[i] in [tgt_words[j] for j in nn_indices[i][:5]] for i in range(len(tgt_words))])

    return precision_at_1, precision_at_5

# Main Code to Run the Full Pipeline

if __name__ == "__main__":
    # Step 1: Load Pre-trained MUSE Embeddings
    print("Loading embeddings...")
    EN_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.en.vec"
    HI_EMBEDDINGS_PATH = "/content/drive/MyDrive/MUSE/wiki.hi.vec"
    LEXICON_PATH = "en-hi.txt"  # Adjust path as needed

    en_embeddings = load_muse_embeddings(EN_EMBEDDINGS_PATH)
    hi_embeddings = load_muse_embeddings(HI_EMBEDDINGS_PATH)

    # Step 2: Load Bilingual Lexicon
    print("Loading bilingual lexicon...")
    lexicon = load_bilingual_lexicon(LEXICON_PATH, max_pairs=10000)

    # Step 3: Extract Embeddings for Words in the Lexicon
    print("Extracting embeddings for alignment...")
    X, Y, valid_pairs = extract_embedding_matrix(lexicon, en_embeddings, hi_embeddings)

    # Step 4: Perform Procrustes Alignment
    print("Performing Procrustes alignment...")
    W_procrustes = procrustes_alignment(X, Y)

    # Step 5: Perform CSLS with Adversarial Training
    print("Performing CSLS with adversarial training...")
    discriminator = adversarial_training(X, Y, epochs=10, batch_size=32, lr=0.001)

    # Evaluate translation accuracy for Procrustes
    print("Evaluating Procrustes translation accuracy...")
    p_at_1_procrustes, p_at_5_procrustes = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, W=W_procrustes, method='procrustes')

    # Evaluate translation accuracy for CSLS
    print("Evaluating CSLS translation accuracy...")
    p_at_1_csls, p_at_5_csls = evaluate_translation(valid_pairs, en_embeddings, hi_embeddings, discriminator=discriminator, method='csls')

    # Print Results
    print(f"Procrustes Precision@1: {p_at_1_procrustes * 100:.2f}%")
    print(f"Procrustes Precision@5: {p_at_5_procrustes * 100:.2f}%")
    print(f"CSLS Precision@1: {p_at_1_csls * 100:.2f}%")
    print(f"CSLS Precision@5: {p_at_5_csls * 100:.2f}%")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin